# Интерполяционный многочлен Лагранжа

$$ L(x)=\sum _{{i=0}}^{n}y_{i}l_{i}(x), $$

где базисные полиномы определяются по формуле:
$$
{\displaystyle l_{i}(x)=\prod _{j=0,j\neq i}^{n}{\frac {x-x_{j}}{x_{i}-x_{j}}}={\frac {x-x_{0}}{x_{i}-x_{0}}}\cdots {\frac {x-x_{i-1}}{x_{i}-x_{i-1}}}\cdot {\frac {x-x_{i+1}}{x_{i}-x_{i+1}}}\cdots {\frac {x-x_{n}}{x_{i}-x_{n}}}}
$$



In [1]:
import numpy as np 
import math

# Вариант 5
x = [0.298, 0.303, 0.310, 0.317, 0.323, 0.330, 0.339]
y = [3.2557, 3.174, 3.1218, 3.0482, 2.9875, 2.9195, 2.8359]

#x1 = 0.314

def interpolate_lagrange(x, y, x1) :
    res = 0.0
    for i in range (0, len(x)):
        p = 1.0
        for j in range (0, len(x)):
            if(i != j):
                p *= (x1 - x[j])/(x[i]-x[j])
        res += p*y[i]
    return res

print(interpolate_lagrange(x, y, 0.314))

3.0811579937758635


# Первая интерполяционная формулой Ньютона для интерполирования вперед

$$P_{n}(x)=y_{0}+t\Delta y_{0}+{\frac  {t(t-1)}{2!}}\Delta ^{2}y_{0}+\ldots +{\frac  {t(t-1)\ldots (t-n+1)}{n!}}\Delta ^{n}y_{0}$$, где ${\displaystyle t={\frac {x-x_{0}}{h}},\;y_{i}=f_{i}} t={\frac  {x-x_{0}}h},\;y_{i}=f_{i}$, а выражения вида ${\displaystyle \Delta ^{k}y_{0}} $ — конечные разности.


In [2]:
# Вариант 5
x = [0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55]
y = [1.2557, 2.1764, 3.1218, 4.0482, 5.9875, 6.9195, 7.8359]

# x2 = 0.253 
# x3 = 0.512

# Конечные разности
def coef(y, n, i):
    if n == 0:
        return (y[i + 1] - y[i]) 
    return (coef(y, n - 1, i + 1) - coef(y, n - 1, i))

# Первая интерполяционная формулой Ньютона для интерполирования вперед
def interpolate_newton_forward(x, y, x0):
    i = next(i - 1 for i, x_i in enumerate(x) if x_i > x0)
    x_i = x[i]
    h = x[1] - x[0]
    t = (x0 - x_i) / h

    return y[i] + sum(np.prod(list(t - k for k in range(j))) / math.factorial(j - 1) * coef(y, j - 1, i)
             for j in range(1, len(x) - 1 - i))

print(interpolate_newton_forward(x, y, 0.253));

1.0315088144385198


# Интерполяционный многочлен Ньютона с разделенными разностями

Формула для разделённой разности: 

$${\displaystyle f(x_{0};\;x_{1};\;\ldots ;\;x_{n})=\sum _{j=0}^{n}{\dfrac {f(x_{j})}{\prod \limits _{i=0 \atop i\neq j}^{n}(x_{j}-x_{i})}},} {\displaystyle f(x_{0};\;x_{1};\;\ldots ;\;x_{n})=\sum _{j=0}^{n}{\dfrac {f(x_{j})}{\prod \limits _{i=0 \atop i\neq j}^{n}(x_{j}-x_{i})}}}$$


$${\displaystyle L_{n}(x)=f(x_{0})+f(x_{0};\;x_{1})\cdot (x-x_{0})+f(x_{0};\;x_{1};\;x_{2})\cdot (x-x_{0})\cdot (x-x_{1})+\ldots +f(x_{0};\;\ldots ;\;x_{n})\cdot \prod _{k=0}^{n-1}(x-x_{k})}$$

In [20]:
# Вариант 5
x = [0.298, 0.303, 0.310, 0.317, 0.323, 0.330, 0.339]
y = [3.2557, 3.174, 3.1218, 3.0482, 2.9875, 2.9195, 2.8359]

#x4 = 0.337

def newton_with_div_diff_st1(x, y, i0, i1):
    return (y[i1] - y[i0]) / (x[i1] - x[i0])

def newton_with_div_diff_st2(x, y, i0, i1, i2):
    return (newton_with_div_diff_st1(x, y, i1, i2) - newton_with_div_diff_st1(x, y, i0, i1)) / (x[i2] - x[i0])

# Интерполяционный многочлен Ньютона с разделенными разностями
def newton_with_div_diff(x, y, x0):
    i0 = 0
    for i in range (0, len(x)):
        if (i == len(x) - 4) or (len(x) - 4 <= 0):
            i0 = len(x) - 4
            break
        if (i == 0) and (x0 <= x[0]):
            break
        if (i != 0) and (x0 <= x[i]) and (x0 > x[i + 1]):
            i0 = i
            break
    i1 = i0 + 1
    i2 = i1 + 1
    res1 = y[i0] + (x0 - x[i0]) * newton_with_div_diff_st1(x, y, i0, i1) + (x0 - x[i0]) * (x0 - x[i1]) * newton_with_div_diff_st2(x, y, i0, i1, i2)
    i0 = i0 + 1
    i1 = i1 + 1
    i2 = i2 + 1
    res2 = y[i0] + (x0 - x[i0]) * newton_with_div_diff_st1(x, y, i0, i1) + (x0 - x[i0]) * (x0 - x[i1]) * newton_with_div_diff_st2(x, y, i0, i1, i2)

    return (res1 + res2)/2

print(newton_with_div_diff(x, y, 0.337)) 

2.854319444444445


In [4]:
def inp(x):
    while (True):
        try:
            x0 = float(input())
            if x0 < min(x) or x0 > max(x):
                print("Внимание: введенное число выходит за границу массива:")
            return x0
        except Exception:
            print("Ошибка. Проверьте правильность введенных данных")


with open('table1.txt', 'r') as myfile:
    content = myfile.readlines()
content = [x.strip() for x in content] 
x1 = [float(x) for x in content[0].split()]
y1 = [float(x) for x in content[1].split()]

with open('table5.txt', 'r') as myfile:
    content = myfile.readlines()
content = [x.strip() for x in content] 
x2 = [float(x) for x in content[0].split()]
y2 = [float(x) for x in content[1].split()]

print(f'Таблица 1:\nx: {x1}\ny: {y1}\n\n')
print(f'Таблица 5:\nx: {x2}\ny: {y2}\n\n')

print("Введите Х0 для поиска приближенного значения функции с помощью многочлена Лагранжа:")
x0 = inp(x1)
print(f'Приближенное значение функции при Х0 = {x0} с помошью многочлена Лагранжа: {interpolate_lagrange(x1, y1, x0)}\n\n')

print("Введите Х0 для поиска приближенного значения функции с помощью первой интерполяционной формулы Ньютона :")
x0 = inp(x2)
print(f'Приблеженное значение функции при Х0 = {x0} с помощью первой интерполяционной формулы Ньютона : {interpolate_newton_forward(x2, y2, x0)}\n\n')

print("Введите Х0 для поиска приближенного значения функции с помощью интерполяционного монгочлена Ньютона с разделенными разностями:")
x0 = inp(x1)
print(f'Приблеженное значение функции при Х0 = {x0} с помощью интерполяционного монгочлена Ньютона с разделенными разностями : {newton_with_div_diff(x1, y1, x0)}\n\n')

Таблица 1:
x: [0.298, 0.303, 0.31, 0.317, 0.323, 0.33, 0.339]
y: [3.2557, 3.174, 3.1218, 3.0482, 2.9875, 2.9195, 2.8359]


Таблица 5:
x: [0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55]
y: [1.2557, 2.1764, 3.1218, 4.0482, 5.9875, 6.9195, 7.8359]


Введите Х0 для поиска приближенного значения функции с помощью многочлена Лагранжа:
0.3
Приближенное значение функции при Х0 = 0.3 с помошью многочлена Лагранжа: 3.2089904348597353


Введите Х0 для поиска приближенного значения функции с помощью первой интерполяционной формулы Ньютона :
0.3
Приблеженное значение функции при Х0 = 0.3 с помощью первой интерполяционной формулы Ньютона : 2.1764


Введите Х0 для поиска приближенного значения функции с помощью интерполяционного монгочлена Ньютона с разделенными разностями:
0.3
Приблеженное значение функции при Х0 = 0.3 с помощью интерполяционного монгочлена Ньютона с разделенными разностями : 3.208990434859736


